In [17]:
# !pip install langchain
# !pip install faiss-cpu
# !pip install pypdf
# !pip install openai
# !pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.5 MB/s eta 0:00:00


In [5]:
import langchain
import pandas as pd

In [6]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader

In [15]:
from langchain.document_loaders import TextLoader,PyPDFLoader

filepath = '/content/202045001.pdf'
filename = filepath.split("/")[-1]
loader = PyPDFLoader(filename)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)
db.save_local(filename)


In [18]:
db = FAISS.from_documents(docs, embeddings)
query = "What is the location of the employer Wide World Importers ?"
docs = db.similarity_search(query)

In [29]:
# docs_and_scores = db.similarity_search_with_score(query)
# docs_and_scores

In [23]:
embedding_vector = embeddings.embed_query(query)
docs_and_scores = db.similarity_search_by_vector(embedding_vector)
docs_and_scores

[Document(page_content="CaseId: 202045001\nEmployer: Wide World Importers\nLocation: Miami \nEvent Date: 1/1/2015\nEvent: Caught in or compressed by equipment or objects, unspeciﬁed \nNature: multiple injuries\nFinal Narrative Hospitalized AmputationPart Of\nBodySource\nAn employee's leg was pinned between a truck and\nthe powered pallet jack being operated. The\nemployee was hospitalized for treatment/surgery at\nNavicent Health.1 0 legs pallet jack-\npowered\nPage 1 of 1", metadata={'source': '/content/202045001.pdf', 'page': 0})]

In [24]:
db.save_local("faiss_index")

In [26]:
new_db = FAISS.load_local("faiss_index", embeddings)

In [27]:
docs = new_db.similarity_search(query)

In [31]:
docs[0].page_content

"CaseId: 202045001\nEmployer: Wide World Importers\nLocation: Miami \nEvent Date: 1/1/2015\nEvent: Caught in or compressed by equipment or objects, unspeciﬁed \nNature: multiple injuries\nFinal Narrative Hospitalized AmputationPart Of\nBodySource\nAn employee's leg was pinned between a truck and\nthe powered pallet jack being operated. The\nemployee was hospitalized for treatment/surgery at\nNavicent Health.1 0 legs pallet jack-\npowered\nPage 1 of 1"

In [ ]:
import openai
from datetime import datetime
openai.api_key = os.environ['OPENAI_API_KEY']  # Replace with your API key

def ask_gpt3(content,question):

    prompt = f"Please answer the following question based on the information provided below. If the answer is not found in the content, please reply with 'don't know'.\n\n{content}\n\nQuestion: {question}"
    response = openai.Completion.create(
        engine="davinci",
        prompt=prompt,
        max_tokens=250,
        n=1,
        stop=None,
        temperature=0.5
    )

    answer = response.choices[0].text.strip()
    return answer
t1 = datetime.now()
answer = ask_gpt3(docs[0].page_content,query)
answer = answer.split("\n")[0]
answer = answer.replace("Answer:","").strip()
t2 = datetime.now()
print(answer,answer.split("\n")[0],t2-t1)